In [1]:
!pip install mlflow

In [2]:
!pip install PyTDC

In [3]:
!pip install scikit-learn

In [4]:
!pip install mlflow matplotlib hyperopt

In [5]:
import mlflow
import mlflow.sklearn
import pandas as pd
import numpy as np
from tdc.multi_pred import TrialOutcome
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

import sys
import warnings
from itertools import cycle

import matplotlib.pyplot as plt
from sklearn import datasets

from sklearn.model_selection import train_test_split

# Import mlflow
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType
from mlflow.store.artifact.runs_artifact_repo import RunsArtifactRepository

# Import hyperopt
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [6]:
%env MLFLOW_TRACKING_SERVER_CERT_PATH=/home/jovyan/work/cert/ca.pem
%env MLFLOW_TRACKING_URI=https://public-tracking.mlflow-e00v0qa5n6ywddys75.backbone-e00m8trsn6mra9a7kk.msp.eu-north1.nebius.cloud
%env MLFLOW_TRACKING_USERNAME=mlops_course_admin
%env MLFLOW_TRACKING_PASSWORD=NAv2VcZafgku5D$
%env MLFLOW_ENABLE_SYSTEM_METRICS_LOGGING=true

env: MLFLOW_TRACKING_SERVER_CERT_PATH=/home/jovyan/work/cert/ca.pem
env: MLFLOW_TRACKING_URI=https://public-tracking.mlflow-e00v0qa5n6ywddys75.backbone-e00m8trsn6mra9a7kk.msp.eu-north1.nebius.cloud
env: MLFLOW_TRACKING_USERNAME=mlops_course_admin
env: MLFLOW_TRACKING_PASSWORD=NAv2VcZafgku5D$
env: MLFLOW_ENABLE_SYSTEM_METRICS_LOGGING=true


In [7]:
from tdc.multi_pred import TrialOutcome
data = TrialOutcome(name = 'phase1') # 'phase2' / 'phase3'
split = data.get_split()

train_data = split["train"]
test_data = split["test"]

#Remove id column
train_data = train_data.drop("nctid",axis=1)
test_data = test_data.drop("nctid",axis=1)

#Remove rows with missing data
train_data = train_data.dropna()
test_data = test_data.dropna()

#Convert date columns to datetime format
train_data['start_date'] = pd.to_datetime(train_data['start_date'])
train_data['complete_date'] = pd.to_datetime(train_data['complete_date'])

test_data['start_date'] = pd.to_datetime(test_data['start_date'])
test_data['complete_date'] = pd.to_datetime(test_data['complete_date'])

#Add a column that contains the duration
train_data['duration'] = (train_data['complete_date'] - train_data['start_date']).dt.days
test_data['duration'] = (test_data['complete_date'] - test_data['start_date']).dt.days

train_data = train_data.drop(['start_date', 'complete_date'], axis=1)
test_data = test_data.drop(['start_date', 'complete_date'], axis=1)

x_duration_train = np.array(train_data["duration"].values)
y_train = np.array(train_data["Y"].values)

x_duration_test = np.array(test_data["duration"].values)
y_test = np.array(test_data["Y"].values)


Found local copy...
Loading...
Done!


In [8]:
# Ajouter une colonne avec le nombre de molécules par SMILES
test_data["num_molecules"] = test_data["drug_molecules"].astype(str).apply(lambda x: len(x.split("__")))
train_data["num_molecules"] = train_data["drug_molecules"].astype(str).apply(lambda x: len(x.split("__")))

# Trouver le maximum
max_molecules_test = test_data["num_molecules"].max()
max_molecules_train = train_data["num_molecules"].max()

# Supprimer les lignes avec un nombre de molécules supérieur au maximum
test_data = test_data.drop("num_molecules", axis=1)
train_data = train_data.drop("num_molecules", axis=1)

max_molecules = max(max_molecules_test, max_molecules_train)
print(max_molecules)

12


In [9]:
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem

n_bits = 2048

def smiles_to_fingerprint(smiles, radius=2, n_bits=n_bits):
    resultat = np.zeros(n_bits*max_molecules)
    smiles = smiles.split("__")
    for i, smile in enumerate(smiles):
      try:
          mol = Chem.MolFromSmiles(smile)
          if mol:
              fingerprint = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
              resultat[i*n_bits:(1+i)*n_bits] = np.array(fingerprint)
          else:
              print(f" Warning: Invalid SMILES '{smiles}'")
      except Exception as e:
          print(f" Error processing SMILES '{smiles}': {e}")
      return resultat

In [10]:
fingerprints_train = np.array([smiles_to_fingerprint(smiles) for smiles in train_data["drug_molecules"].astype(str)])
fingerprints_test = np.array([smiles_to_fingerprint(smiles) for smiles in test_data["drug_molecules"].astype(str)])

x_molecules_train = np.stack(fingerprints_train)
x_molecules_test = np.stack(fingerprints_test)

In [11]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
import torch
from torch.nn.utils.rnn import pad_sequence

# Combine both datasets for encoding
combined_icd_codes = pd.concat([train_data["icdcodes"], test_data["icdcodes"]]).astype(str)

# Extract ICD-10 codes and split multiple entries
icd_codes_combined = combined_icd_codes.apply(lambda x: x.split('__'))

# Get all unique ICD-10 codes
unique_codes = list(set([code for sublist in icd_codes_combined for code in sublist]))

# Fit LabelEncoder on ALL unique codes
encoder = LabelEncoder()
encoder.fit(unique_codes)


# Encode training ICD-10 codes
encoded_icd_codes_train = train_data["icdcodes"].astype(str).apply(lambda x: encoder.transform(x.split('__')))

# Encode test ICD-10 codes
encoded_icd_codes_test = test_data["icdcodes"].astype(str).apply(lambda x: encoder.transform(x.split('__')))


# Determine the maximum sequence length
max_length_train = max(encoded_icd_codes_train.apply(len))
max_length_test = max(encoded_icd_codes_test.apply(len))

max_length = max(max_length_train, max_length_test)


# Create a padded tensor with zeros (padding short sequences)
padded_codes_train = torch.zeros((len(train_data), max_length), dtype=torch.long)
padded_codes_test = torch.zeros((len(test_data), max_length), dtype=torch.long)

for i, codes in enumerate(encoded_icd_codes_train):
    padded_codes_train[i, :len(codes)] = torch.tensor(codes, dtype=torch.long)

for i, codes in enumerate(encoded_icd_codes_test):
    padded_codes_test[i, :len(codes)] = torch.tensor(codes, dtype=torch.long)

# Convert to a NumPy array (optional for ML models like RandomForest)
x_icd_train = padded_codes_train.numpy()  # (num_samples, max_length)
x_icd_test = padded_codes_test.numpy()  # (num_samples, max_length)


In [12]:
from sklearn.model_selection import train_test_split

# Ensure all inputs have the same number of dimensions
x_duration_train = x_duration_train.reshape(-1, 1)  # Convert 1D to 2D
x_duration_test = x_duration_test.reshape(-1, 1)  # Convert 1D to 2D

# Concatenate all features
X_train = np.hstack([x_duration_train, x_molecules_train, x_icd_train])  # Final Feature Matrix (num_samples, total_features)
X_test = np.hstack([x_duration_test, x_molecules_test, x_icd_test])  # Final Feature Matrix (num_samples, total_features)

# Convert to PyTorch tensors (for deep learning)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)



In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Initialize and train the model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predictions
y_pred = rf_model.predict(X_test)

# Evaluate performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Random Forest Accuracy: {accuracy:.4f}")

Random Forest Accuracy: 0.5759


In [14]:
search_space = space = {
    'n_estimators': hp.choice('n_estimators', [10, 50, 100, 200, 500,600,800]),
    'random_state': hp.choice('random_state', [0, 7, 42])
}

In [15]:
def objective (params):
    with mlflow.start_run(run_name=f"Run_{params['n_estimators']}_{params['random_state']}"):
        mlflow.set_tag("model", "RandomForest")
        mlflow.log_params(params)
        # Train Elasticnet
        lr = RandomForestClassifier(**params)
        lr.fit(X_train, y_train)
        y_pred = lr.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        #accuracy=accuracy_score(y_test,y_pred)
        mlflow.log_metric("accuracy_score", accuracy)
        # Infer model signature
        predictions = lr.predict(X_train)
        signature = infer_signature(X_train, predictions)

        mlflow.sklearn.log_model(lr, "model", signature=signature)

    return {'loss': -accuracy, 'status': STATUS_OK }



In [ ]:
mlflow.set_experiment("RandomForestClassifier-AAMSS")
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=75,
    trials=Trials()
)

2025/01/29 21:41:29 INFO mlflow.tracking.fluent: Experiment with name 'RandomForestClassifier-AAMSS' does not exist. Creating a new experiment.


  0%|          | 0/75 [00:00<?, ?trial/s, best loss=?]

2025/01/29 21:41:29 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet


2025/01/29 21:41:30 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics beca

🏃 View run Run_100_42 at: https://public-tracking.mlflow-e00v0qa5n6ywddys75.backbone-e00m8trsn6mra9a7kk.msp.eu-north1.nebius.cloud/#/experiments/12/runs/71924eb2b0bd44a09f1534ba9d3ffa41

🧪 View experiment at: https://public-tracking.mlflow-e00v0qa5n6ywddys75.backbone-e00m8trsn6mra9a7kk.msp.eu-north1.nebius.cloud/#/experiments/12

  0%|          | 0/75 [00:35<?, ?trial/s, best loss=?]

2025/01/29 21:42:05 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...

2025/01/29 21:42:06 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!



  1%|▏         | 1/75 [00:36<45:12, 36.66s/trial, best loss: -0.5759493670886076]

2025/01/29 21:42:06 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: Failed to initialize NVML, skip logging GPU metrics: NVML Shared Library Not Found.

2025/01/29 21:42:06 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.

